In [1]:
from ortools.linear_solver import pywraplp
import seaborn as sns
import numpy as np
import pandas as pd
import time
from models import problem_setup.setup

## Model Implementation Empirical Time Assessment

In this section, we proceed to study how different algorithms and solvers succeed in finding a solution to our problem with our model implementation.

In [ ]:
parameters = pywraplp.MPSolverParameters()

#### Solver/Optimizer Analysis

Despite not proving the solver in practical lessons, we include solvers like __Gurobi__, __XPRESS__ or __CPLEX__ in their suitable versions for MILP problems.

In [ ]:
solver_params = {}

solver_types = ["GLOP","SCIP","GUROBI","CPLEX","XPRESS"]
for algorithm in solver_types:
    solver = pywraplp.Solver.CreateSolver(algorithm)
    
    # Create function as a wraper that takes a solver defined in the previous line
    # and defines the variables and 
    
    

In [6]:
pywraplp.MPSolverParameters()

<ortools.linear_solver.pywraplp.MPSolverParameters; proxy of <Swig Object of type 'operations_research::MPSolverParameters *' at 0x000001DFC15BE010> >